In [1]:
import sys
sys.path.append("/kaggle/input/fu-net/FuNET-C")

In [2]:
import numpy as np
import scipy.io as scio
import tensorflow as tf
from tf_utils import random_mini_batches_GCN1, convert_to_one_hot
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create logistic‐regression probe once
probe_clf = LogisticRegression(max_iter=2000)

# TF1.x setup
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

# --- Placeholders ---
spec_ph   = tf.compat.v1.placeholder(tf.float32, [None, 200],    name="spec")
patch_ph  = tf.compat.v1.placeholder(tf.float32, [None, 7*7*200], name="patch")
label_ph  = tf.compat.v1.placeholder(tf.float32, [None, 16],     name="label")
lap_ph    = tf.compat.v1.placeholder(tf.float32, [None, None],   name="lap")

# For SimCLR
spec1_ph, spec2_ph = [
    tf.compat.v1.placeholder(tf.float32, [None, 200], name=n)
    for n in ("spec1","spec2")
]
patch1_ph, patch2_ph = [
    tf.compat.v1.placeholder(tf.float32, [None, 7*7*200], name=n)
    for n in ("patch1","patch2")
]

# --- FuNet-C variables ---
with tf.compat.v1.variable_scope("FUNET"):
    Wg = tf.compat.v1.get_variable("x_w1",    [200,128], initializer=tf.keras.initializers.GlorotUniform())
    bg = tf.compat.v1.get_variable("x_b1",    [128],    initializer=tf.zeros_initializer())
    Wc1 = tf.compat.v1.get_variable("x_conv_w1",[3,3,200,32], initializer=tf.keras.initializers.GlorotUniform())
    bc1 = tf.compat.v1.get_variable("x_conv_b1",[32],       initializer=tf.zeros_initializer())
    Wc2 = tf.compat.v1.get_variable("x_conv_w2",[3,3,32,64],  initializer=tf.keras.initializers.GlorotUniform())
    bc2 = tf.compat.v1.get_variable("x_conv_b2",[64],       initializer=tf.zeros_initializer())
    Wc3 = tf.compat.v1.get_variable("x_conv_w3",[1,1,64,128],initializer=tf.keras.initializers.GlorotUniform())
    bc3 = tf.compat.v1.get_variable("x_conv_b3",[128],      initializer=tf.zeros_initializer())
    Wf1 = tf.compat.v1.get_variable("x_jw1",[256,128],       initializer=tf.keras.initializers.GlorotUniform())
    bf1 = tf.compat.v1.get_variable("x_jb1",[128],           initializer=tf.zeros_initializer())
    Wf2 = tf.compat.v1.get_variable("x_jw2",[128,16],        initializer=tf.keras.initializers.GlorotUniform())
    bf2 = tf.compat.v1.get_variable("x_jb2",[16],            initializer=tf.zeros_initializer())

# --- Encoder ---
def funet_encode(spec, patch, lap):
    h = tf.matmul(spec, Wg) + bg
    g = tf.matmul(lap, h)
    r  = tf.reshape(patch, [-1,7,7,200])
    c1 = tf.nn.relu(tf.nn.max_pool2d(tf.nn.conv2d(r,Wc1,[1,1,1,1],"SAME")+bc1,2,2,"SAME"))
    c2 = tf.nn.relu(tf.nn.max_pool2d(tf.nn.conv2d(c1,Wc2,[1,1,1,1],"SAME")+bc2,2,2,"SAME"))
    c3 = tf.nn.relu(tf.nn.max_pool2d(tf.nn.conv2d(c2,Wc3,[1,1,1,1],"SAME")+bc3,2,2,"SAME"))
    c  = tf.reshape(c3,[-1,128])
    j  = tf.concat([g,c],1)
    f1 = tf.nn.relu(tf.matmul(j,Wf1)+bf1)
    logits = tf.matmul(f1,Wf2)+bf2
    return logits, j

# --- Supervised head ---
logits, joint_feats = funet_encode(spec_ph, patch_ph, lap_ph)
sup_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label_ph)
)
sup_opt = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(sup_loss)
sup_acc = tf.reduce_mean(tf.cast(
    tf.equal(tf.argmax(logits,1), tf.argmax(label_ph,1)), tf.float32
))

# --- SimCLR augmentations ---
a1 = spec1_ph + tf.random.normal(tf.shape(spec1_ph), stddev=0.01)
a2 = spec2_ph + tf.random.normal(tf.shape(spec2_ph), stddev=0.01)
def aug_patch(ph):
    b = tf.shape(ph)[0]
    r = tf.reshape(ph, [b,7,7,200])
    cr= tf.image.random_crop(r,[b,5,5,200])
    cr= tf.image.resize(cr,[7,7])+tf.random.normal(tf.shape(r),stddev=0.01)
    return tf.reshape(cr,[b,-1])
ap1 = aug_patch(patch1_ph)
ap2 = aug_patch(patch2_ph)

# Reuse FUNET
with tf.compat.v1.variable_scope("FUNET", reuse=True):
    pass

_, h1 = funet_encode(a1, ap1, lap_ph)
_, h2 = funet_encode(a2, ap2, lap_ph)

# --- Projection head variables ---
with tf.compat.v1.variable_scope("SIMCLR_HEAD"):
    # two-layer MLP
    Wp1 = tf.compat.v1.get_variable("Wp1", [256,256], initializer=tf.keras.initializers.GlorotUniform())
    bp1 = tf.compat.v1.get_variable("bp1", [256], initializer=tf.zeros_initializer())
    Wp2 = tf.compat.v1.get_variable("Wp2", [256,128], initializer=tf.keras.initializers.GlorotUniform())
    bp2 = tf.compat.v1.get_variable("bp2", [128], initializer=tf.zeros_initializer())

def project(h):
    x = tf.nn.relu(tf.matmul(h, Wp1) + bp1)
    return tf.matmul(x, Wp2) + bp2

z1 = project(h1)
z2 = project(h2)

# --- NT-Xent loss ---
z1n = tf.math.l2_normalize(z1,1)
z2n = tf.math.l2_normalize(z2,1)
sim  = tf.matmul(z1n, z2n, transpose_b=True)
labels_con = tf.range(tf.shape(sim)[0])
sim_loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=sim, labels=labels_con)
)

# --- SimCLR optimizer on just projection head ---
proj_vars = [Wp1, bp1, Wp2, bp2]
sim_opt = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(sim_loss, var_list=proj_vars)

# --- Run session ---
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    # Load data
    X_tr = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Train_X.mat')['Train_X']
    P_tr = scio.loadmat('/kaggle/input/fu-net/FuNET-C/X_train.mat')['X_train']
    L_tr = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Train_L.mat')['Train_L']
    Ytr  = convert_to_one_hot(scio.loadmat('/kaggle/input/fu-net/FuNET-C/TrLabel.mat')['TrLabel']-1,16).T

    X_te = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Test_X.mat')['Test_X']
    P_te = scio.loadmat('/kaggle/input/fu-net/FuNET-C/X_test.mat')['X_test']
    L_te = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Test_L.mat')['Test_L']
    Yte  = convert_to_one_hot(scio.loadmat('/kaggle/input/fu-net/FuNET-C/TeLabel.mat')['TeLabel']-1,16).T

    # Supervised
    for e in range(1,201):
        mbs = random_mini_batches_GCN1(X_tr, P_tr, Ytr, L_tr, 32, e)
        for bx,bp,by,bl in mbs:
            sess.run(sup_opt, feed_dict={spec_ph:bx, patch_ph:bp, label_ph:by, lap_ph:bl})
        if e%50==0:
            tr_acc = sess.run(sup_acc, feed_dict={spec_ph:X_tr, patch_ph:P_tr, label_ph:Ytr, lap_ph:L_tr})
            te_acc = sess.run(sup_acc, feed_dict={spec_ph:X_te, patch_ph:P_te, label_ph:Yte, lap_ph:L_te})
            print(f"[Sup] {e}: Train={tr_acc:.4f}, Test={te_acc:.4f}")

    for epoch in range(1, 201):
        # 1) SimCLR training
        idx = np.random.permutation(X_tr.shape[0])
        for i in range(X_tr.shape[0] // 32):
            bi = idx[i*32:(i+1)*32]
            sess.run(sim_opt, feed_dict={
                spec1_ph: X_tr[bi],
                spec2_ph: X_tr[bi],
                patch1_ph: P_tr[bi],
                patch2_ph: P_tr[bi],
                lap_ph:     L_tr[np.ix_(bi, bi)]
            })
    
        # Only do probe at epochs 50,100,150,200
        if epoch % 50 == 0:
            # 2) Extract features
            F_tr = sess.run(z1n, feed_dict={spec1_ph: X_tr, patch1_ph: P_tr, lap_ph: L_tr})
            F_te = sess.run(z1n, feed_dict={spec1_ph: X_te, patch1_ph: P_te, lap_ph: L_te})
    
            # 3) Fit & evaluate
            y_tr = np.argmax(Ytr, axis=1)
            y_te = np.argmax(Yte, axis=1)
    
            probe_clf.fit(F_tr, y_tr)
            tr_acc = probe_clf.score(F_tr, y_tr)
            te_acc = probe_clf.score(F_te, y_te)
    
            print(f"[SimCLR] Epoch {epoch}: Probe Train Acc = {tr_acc:.4f}, Probe Test Acc = {te_acc:.4f}")


2025-06-28 04:38:00.709186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751085480.906918      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751085480.958752      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1751085494.681357      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1751085494.706445      35 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
I0000 00:00:1751085505.831693      94 cuda_dnn.cc:529] Loaded cuDNN version 90300


[Sup] 50: Train=0.9007, Test=0.7070
[Sup] 100: Train=0.9871, Test=0.7959
[Sup] 150: Train=1.0000, Test=0.8246
[Sup] 200: Train=1.0000, Test=0.8197
[SimCLR] Epoch 50: Probe Train Acc = 0.5482, Probe Test Acc = 0.4976
[SimCLR] Epoch 100: Probe Train Acc = 0.5799, Probe Test Acc = 0.5069
[SimCLR] Epoch 150: Probe Train Acc = 0.6072, Probe Test Acc = 0.5184
[SimCLR] Epoch 200: Probe Train Acc = 0.6129, Probe Test Acc = 0.5250


In [3]:
import numpy as np
import scipy.io as scio
import tensorflow as tf
from tf_utils import random_mini_batches_GCN1, convert_to_one_hot

# TF1.x setup
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

spec_ph   = tf.compat.v1.placeholder(tf.float32, [None, 200],    name="spec")
patch_ph  = tf.compat.v1.placeholder(tf.float32, [None, 7*7*200], name="patch")
label_ph  = tf.compat.v1.placeholder(tf.float32, [None, 16],     name="label")
lap_ph    = tf.compat.v1.placeholder(tf.float32, [None, None],   name="lap")

# For SimCLR augmentations
spec1_ph, spec2_ph = [
    tf.compat.v1.placeholder(tf.float32, [None,200], name=n)
    for n in ("spec1","spec2")
]
patch1_ph, patch2_ph = [
    tf.compat.v1.placeholder(tf.float32, [None,7*7*200], name=n)
    for n in ("patch1","patch2")
]

with tf.compat.v1.variable_scope("FUNET"):
    # GCN
    Wg  = tf.compat.v1.get_variable("x_w1",    [200,128], initializer=tf.keras.initializers.GlorotUniform())
    bg  = tf.compat.v1.get_variable("x_b1",    [128],    initializer=tf.zeros_initializer())
    # CNN
    Wc1 = tf.compat.v1.get_variable("x_conv_w1",[3,3,200,32], initializer=tf.keras.initializers.GlorotUniform())
    bc1 = tf.compat.v1.get_variable("x_conv_b1",[32],       initializer=tf.zeros_initializer())
    Wc2 = tf.compat.v1.get_variable("x_conv_w2",[3,3,32,64],  initializer=tf.keras.initializers.GlorotUniform())
    bc2 = tf.compat.v1.get_variable("x_conv_b2",[64],       initializer=tf.zeros_initializer())
    Wc3 = tf.compat.v1.get_variable("x_conv_w3",[1,1,64,128],initializer=tf.keras.initializers.GlorotUniform())
    bc3 = tf.compat.v1.get_variable("x_conv_b3",[128],      initializer=tf.zeros_initializer())
    # Fusion MLP
    Wf1 = tf.compat.v1.get_variable("x_jw1",[256,128],       initializer=tf.keras.initializers.GlorotUniform())
    bf1 = tf.compat.v1.get_variable("x_jb1",[128],           initializer=tf.zeros_initializer())
    Wf2 = tf.compat.v1.get_variable("x_jw2",[128,16],        initializer=tf.keras.initializers.GlorotUniform())
    bf2 = tf.compat.v1.get_variable("x_jb2",[16],            initializer=tf.zeros_initializer())

# --- Encoder function ---
def funet_encode(spec, patch, lap):
    # GCN branch
    h = tf.matmul(spec, Wg) + bg
    g = tf.matmul(lap, h)                # [batch,128]
    # CNN branch
    r  = tf.reshape(patch, [-1,7,7,200])
    c1 = tf.nn.relu(tf.nn.max_pool2d(tf.nn.conv2d(r,Wc1,[1,1,1,1],"SAME")+bc1,2,2,"SAME"))
    c2 = tf.nn.relu(tf.nn.max_pool2d(tf.nn.conv2d(c1,Wc2,[1,1,1,1],"SAME")+bc2,2,2,"SAME"))
    c3 = tf.nn.relu(tf.nn.max_pool2d(tf.nn.conv2d(c2,Wc3,[1,1,1,1],"SAME")+bc3,2,2,"SAME"))
    c  = tf.reshape(c3, [-1,128])        # [batch,128]
    # Fusion
    j  = tf.concat([g,c], axis=1)        # [batch,256]
    f1 = tf.nn.relu(tf.matmul(j,Wf1) + bf1)
    logits = tf.matmul(f1,Wf2) + bf2
    return logits, j

# Augmentations
a1 = spec1_ph + tf.random.normal(tf.shape(spec1_ph), stddev=0.01)
a2 = spec2_ph + tf.random.normal(tf.shape(spec2_ph), stddev=0.01)
def aug_patch(ph):
    b = tf.shape(ph)[0]
    r = tf.reshape(ph,[b,7,7,200])
    cr= tf.image.random_crop(r,[b,5,5,200])
    cr= tf.image.resize(cr,[7,7]) + tf.random.normal(tf.shape(r),stddev=0.01)
    return tf.reshape(cr,[b,-1])
ap1, ap2 = aug_patch(patch1_ph), aug_patch(patch2_ph)

# Reuse FUNET to encode
with tf.compat.v1.variable_scope("FUNET", reuse=True):
    pass
_, h1 = funet_encode(a1, ap1, lap_ph)
_, h2 = funet_encode(a2, ap2, lap_ph)

# Projection MLP under its own scope
with tf.compat.v1.variable_scope("SIMCLR_HEAD"):
    Wp1 = tf.compat.v1.get_variable("Wp1",[256,256], initializer=tf.keras.initializers.GlorotUniform())
    bp1 = tf.compat.v1.get_variable("bp1",[256], initializer=tf.zeros_initializer())
    Wp2 = tf.compat.v1.get_variable("Wp2",[256,128], initializer=tf.keras.initializers.GlorotUniform())
    bp2 = tf.compat.v1.get_variable("bp2",[128], initializer=tf.zeros_initializer())

def project(h):
    x = tf.nn.relu(tf.matmul(h,Wp1) + bp1)
    return tf.matmul(x,Wp2) + bp2

z1, z2 = project(h1), project(h2)

# NT-Xent loss
z1n = tf.math.l2_normalize(z1,1)
z2n = tf.math.l2_normalize(z2,1)
sim  = tf.matmul(z1n, z2n, transpose_b=True)
labels_sim = tf.range(tf.shape(sim)[0])
sim_loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=sim, labels=labels_sim)
)

# SimCLR optimizer on only projection head
proj_vars = [Wp1,bp1,Wp2,bp2]
sim_opt = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(sim_loss, var_list=proj_vars)

# We'll reuse funet_encode on spec_ph/patch_ph
logits, joint_feats = funet_encode(spec_ph, patch_ph, lap_ph)
sup_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label_ph)
)
sup_opt = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(sup_loss)
sup_acc = tf.reduce_mean(tf.cast(
    tf.equal(tf.argmax(logits,1), tf.argmax(label_ph,1)), tf.float32
))


X_tr = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Train_X.mat')['Train_X']
P_tr = scio.loadmat('/kaggle/input/fu-net/FuNET-C/X_train.mat')['X_train']
L_tr = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Train_L.mat')['Train_L']
Ytr  = convert_to_one_hot(scio.loadmat('/kaggle/input/fu-net/FuNET-C/TrLabel.mat')['TrLabel']-1,16).T

X_te = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Test_X.mat')['Test_X']
P_te = scio.loadmat('/kaggle/input/fu-net/FuNET-C/X_test.mat')['X_test']
L_te = scio.loadmat('/kaggle/input/fu-net/FuNET-C/Test_L.mat')['Test_L']
Yte  = convert_to_one_hot(scio.loadmat('/kaggle/input/fu-net/FuNET-C/TeLabel.mat')['TeLabel']-1,16).T

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    # 1) SimCLR pre-training
    for epoch in range(1, 201):
        idx = np.random.permutation(X_tr.shape[0])
        for i in range(X_tr.shape[0]//32):
            bi = idx[i*32:(i+1)*32]
            sess.run(sim_opt, feed_dict={
                spec1_ph: X_tr[bi], spec2_ph: X_tr[bi],
                patch1_ph:P_tr[bi], patch2_ph:P_tr[bi],
                lap_ph:    L_tr[np.ix_(bi,bi)]
            })
        if epoch % 50 == 0:
            loss_val = sess.run(sim_loss, feed_dict={
                spec1_ph: X_tr[bi], spec2_ph: X_tr[bi],
                patch1_ph:P_tr[bi], patch2_ph:P_tr[bi],
                lap_ph:    L_tr[np.ix_(bi,bi)]
            })
            print(f"[SimCLR] Epoch {epoch}, Loss={loss_val:.4f}")

    # 2) Fine-tune end-to-end supervised
    for epoch in range(1, 201):
        mbs = random_mini_batches_GCN1(X_tr, P_tr, Ytr, L_tr, 32, epoch)
        for bx,bp,by,bl in mbs:
            sess.run(sup_opt, feed_dict={
                spec_ph: bx, patch_ph: bp, label_ph: by, lap_ph: bl
            })
        if epoch % 50 == 0:
            tr_acc = sess.run(sup_acc, feed_dict={
                spec_ph: X_tr, patch_ph: P_tr,
                label_ph: Ytr,  lap_ph: L_tr
            })
            te_acc = sess.run(sup_acc, feed_dict={
                spec_ph: X_te, patch_ph: P_te,
                label_ph: Yte,  lap_ph: L_te
            })
            print(f"[Fine-Tune] Epoch {epoch}: Train Acc={tr_acc:.4f}, Test Acc={te_acc:.4f}")


I0000 00:00:1751085646.063625      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


[SimCLR] Epoch 50, Loss=2.5703
[SimCLR] Epoch 100, Loss=2.6167
[SimCLR] Epoch 150, Loss=2.5745
[SimCLR] Epoch 200, Loss=2.5675
[Fine-Tune] Epoch 50: Train Acc=0.9194, Test Acc=0.7329
[Fine-Tune] Epoch 100: Train Acc=0.9986, Test Acc=0.8042
[Fine-Tune] Epoch 150: Train Acc=1.0000, Test Acc=0.8175
[Fine-Tune] Epoch 200: Train Acc=1.0000, Test Acc=0.8106
